O conjunto de dados contém dados fictícios do número de passageiros de avião de 1949 a 1960.
<p>Vamos usar o algoritmo ARIMA para prever o número de passageiros para os próximos 12 meses.</p>


1. Importando as bibliotecas que serão usadas

In [1]:
import pandas as pd
import numpy as np
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_squared_log_error
import warnings
warnings.filterwarnings('ignore')

2. Carregar o conjunto de dados e exibr os primeiros registros

In [2]:
df = pd.read_csv('AirPassengers.csv')
df.head()

,Month,#Passengers
0,1949-01,112
1,1949-02,118
2,1949-03,132
3,1949-04,129
4,1949-05,121


3. Verificar o tipo de dados das colunas

In [3]:
df.dtypes

Month          object
#Passengers     int64
dtype: object

Como se trata de uma série temporal é necessário transformar a coluna Month para objeto datetime

In [4]:
df['Month'] = pd.to_datetime(df['Month'])

Verificando novamente o tipo de dados das colunas

In [5]:
df.dtypes

Month          datetime64[ns]
#Passengers             int64
dtype: object

Podemos notar que o atributo Month foi transformado corretamente

4. Definir a coluna Month como índice
<p>Definir a coluna Month como índice porque os modelos de séries temporais, como ARIMA, assumem que seus dados são indexados por uma variável de tempo.</p>
<p>Isso permite que o modelo acesse e manipule os dados de maneira eficiente e também permite que façamos consultas e operações baseadas no tempo, como agrupar por ano ou calcular a média móvel.</p>

In [6]:
df.set_index('Month', inplace=True)

5. Definindo o modelo ARIMA

In [7]:
model = ARIMA(df['#Passengers'], order=(5, 1, 0))

6. Ajustando o modelo

In [8]:
model_fit = model.fit()

7. Previsão

In [9]:
forecast = model_fit.forecast(steps=12)[0]
print(forecast)

489.4597237481947


8. Avaliação do modelo

In [10]:
y_true = df['#Passengers'].values
y_pred = model_fit.fittedvalues.values

8.1 Calculando as métricas

In [11]:
mse = mean_squared_error(y_true, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_true, y_pred)
mape = np.mean(np.abs((y_true, y_pred) / y_pred)) * 100

8.2 Exibindo as métricas de avaliação

In [12]:
print(f'Erro quadrático médio (MSE): {mse}')
print(f'Raiz do erro quadrático médio (RMSE): {rmse}')
print(f'Erro absoluto médio (MAE): {mae}')
print(f'Erro percentual absoluto médio (MAPE): {mape}')

Erro quadrático médio (MSE): 974.8568871358182
Raiz do erro quadrático médio (RMSE): 31.222698268019986
Erro absoluto médio (MAE): 24.418708447692822
Erro percentual absoluto médio (MAPE): nan


Como o MAPE resultou nan, isso quer dizer que deve ter 0 nos dados, vamos adicionar um pequeno valor aos valores previstos para evitar dividir por zero

In [13]:
y_pred = [max(value, 0.01) for value in y_pred]

Agora vamos usar o Erro Quadrático Médio Percentual (Mean Squared Logarithmic Error - MSLE). O MSLE só considera a proporção entre o valor verdadeiro e o valor previsto, em vez da diferença absoluta entre eles. Portanto, é mais robusto quando se lida com valores de previsão que podem ser zero. E quanto mais próximo de 0 for o MSLE, melhor o modelo.

In [14]:
msle = mean_squared_log_error(y_true, y_pred)
print(f'MSLE: {msle:.4f}')

MSLE: 0.1639
